In [3]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Code/separated_sentences.csv')

# Display the first few rows to check the data
print(df.head())

# Check for missing values
print(df.isnull().sum())

# Drop rows with missing values if any
df.dropna(inplace=True)

# Ensure columns are properly named for clarity
df.columns = ['English', 'Bangla']

# Optional: Check for any duplicates and remove them
df.drop_duplicates(inplace=True)

# Reset index after dropping rows
df.reset_index(drop=True, inplace=True)

print(df.head())


                      English                  Bangla
0       I want to drink water         আমি জল খেতে চাই
1        Sudip is a good boy.   সুদীপ একজন ভালো ছেলে।
2    Think I'll go to school.  ভাবছি আজ স্কুল এ যাবো।
3             My hair is torn    আমার চুল ছেঁড়া গেলো
4  Tumi football khelte paro?  তুমি ফুটবল খেলতে পারো?
English    20
Bangla     20
dtype: int64
                      English                  Bangla
0       I want to drink water         আমি জল খেতে চাই
1        Sudip is a good boy.   সুদীপ একজন ভালো ছেলে।
2    Think I'll go to school.  ভাবছি আজ স্কুল এ যাবো।
3             My hair is torn    আমার চুল ছেঁড়া গেলো
4  Tumi football khelte paro?  তুমি ফুটবল খেলতে পারো?


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
from transformers import pipeline

# Load a pre-trained translation pipeline
translation_pipeline = pipeline("translation_bn_to_en", model="Helsinki-NLP/opus-mt-bn-en")

# Example: Translate a single sentence
translation = translation_pipeline("আপনি কেমন আছেন?")[0]['translation_text']
print("Translated sentence:", translation)

translation = translation_pipeline("আজ আকাশে মেঘ করেছে।")[0]['translation_text']
print("Translated sentence:", translation)

translation = translation_pipeline("আমি প্রতিদিন সকালে ঘুম থেকে উঠি।")[0]['translation_text']
print("Translated sentence:", translation)

translation = translation_pipeline("তোমার নাম কি?")[0]['translation_text']
print("Translated sentence:", translation)


Translated sentence: How are you?
Translated sentence: It's clouds in the sky today.
Translated sentence: I wake up every morning.
Translated sentence: What's your name?


In [1]:
!pip install sacrebleu
!pip install datasets
!pip install bert_score
!pip install transformers
!pip install rouge_score
!pip install sentence-separator
!pip install huggingface_hub

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ee25f7f17c97545cf418a86a9280270976b767c7e86b59fc45ad26bc9f96b146
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
ERROR: Could not find a version that satisfies the requirement sentence-separator (from versions: none)
ERROR: No matching distribution found for sentence-separator


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Traceback (most recent call last):

In [7]:
import pandas as pd
import sacrebleu
from datasets import load_metric
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import bert_score
import torch
from tqdm import tqdm
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Check if a GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load your dataset
df = pd.read_csv(r"/content/drive/MyDrive/Code/separated_sentences.csv")

# Ensure 'English' and 'Bangla' columns are strings
df['English'] = df['English'].astype(str)
df['Bangla'] = df['Bangla'].astype(str)

# Initialize evaluation metrics
rouge = load_metric("rouge")
meteor = load_metric("meteor")
bert_scorer = bert_score.BERTScorer(
    lang="bn", rescale_with_baseline=False, device=device
)

# List of translation models to evaluate
models_to_evaluate = [
    {"name": "facebook/mbart-large-50-many-to-many-mmt", "src_lang": "en_XX", "tgt_lang": "bn_IN"},
    {"name": "Helsinki-NLP/opus-mt-en-bn", "src_lang": None, "tgt_lang": None},
    {"name": "facebook/nllb-200-distilled-600M", "src_lang": "eng_Latn", "tgt_lang": "ben_Beng"},
    {"name": "google/mt5-small", "src_lang": None, "tgt_lang": None},
    {"name": "ai4bharat/indictrans-v2-en-bn", "src_lang": None, "tgt_lang": None},
    {"name": "ai4bharat/indic-bert", "src_lang": None, "tgt_lang": None}

    # Add more models as needed
]

# Initialize a list to store the results
results = []

# Function for batch processing with real-time scoring
def batch_translate_and_score(translator, texts, references, batch_size=16):
    translations = []
    total_bleu = 0
    total_rouge_l = 0
    total_meteor = 0
    total_bert_score_f1 = 0

    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i + batch_size]
        batch_refs = references[i:i + batch_size]
        batch_texts = [str(text) for text in batch_texts]
        translated_batch = translator(batch_texts)
        batch_translations = [item['translation_text'] for item in translated_batch]
        translations.extend(batch_translations)

        # Calculate scores for this batch
        bleu = sacrebleu.corpus_bleu(batch_translations, [batch_refs]).score
        rouge_score = rouge.compute(predictions=batch_translations, references=batch_refs)
        meteor_score = meteor.compute(predictions=batch_translations, references=batch_refs)
        bert_score_res = bert_scorer.score(batch_translations, batch_refs)

        total_bleu += bleu
        total_rouge_l += rouge_score['rougeL'].mid.fmeasure
        total_meteor += meteor_score['meteor']
        total_bert_score_f1 += bert_score_res[2].mean().item()

        # Print scores for this batch
        print(f"\nBatch {i//batch_size + 1} Scores:")
        print(f"BLEU: {bleu:.2f}")
        print(f"ROUGE-L: {rouge_score['rougeL'].mid.fmeasure:.2f}")
        print(f"METEOR: {meteor_score['meteor']:.2f}")
        print(f"BERTScore F1: {bert_score_res[2].mean().item():.2f}")

    # Calculate average scores
    num_batches = len(texts) // batch_size + (1 if len(texts) % batch_size != 0 else 0)
    avg_bleu = total_bleu / num_batches
    avg_rouge_l = total_rouge_l / num_batches
    avg_meteor = total_meteor / num_batches
    avg_bert_score_f1 = total_bert_score_f1 / num_batches

    return translations, avg_bleu, avg_rouge_l, avg_meteor, avg_bert_score_f1

# Iterate over the models
for model_info in models_to_evaluate:
    model_name = model_info['name']
    src_lang = model_info['src_lang']
    tgt_lang = model_info['tgt_lang']

    print(f"\nProcessing model: {model_name}")

    try:
        # Initialize the translation pipeline with GPU support
        if src_lang and tgt_lang:
            translator = pipeline("translation", model=model_name,
                                  src_lang=src_lang, tgt_lang=tgt_lang,
                                  device=0 if torch.cuda.is_available() else -1)
        else:
            translator = pipeline("translation", model=model_name,
                                  device=0 if torch.cuda.is_available() else -1)

        # Translate English sentences to Bengali in batches and get scores
        translations, avg_bleu, avg_rouge_l, avg_meteor, avg_bert_score_f1 = batch_translate_and_score(
            translator, df['English'].tolist(), df['Bangla'].tolist()
        )

        # Store the results
        results.append({
            "Model": model_name,
            "BLEU": avg_bleu,
            "ROUGE_L": avg_rouge_l,
            "METEOR": avg_meteor,
            "BERTScore_F1": avg_bert_score_f1
        })

        print(f"\nFinal Scores for {model_name}:")
        print(f"Average BLEU: {avg_bleu:.2f}")
        print(f"Average ROUGE-L: {avg_rouge_l:.2f}")
        print(f"Average METEOR: {avg_meteor:.2f}")
        print(f"Average BERTScore F1: {avg_bert_score_f1:.2f}")

    except Exception as e:
        print(f"Error processing model {model_name}: {str(e)}")
        results.append({
            "Model": model_name,
            "Error": str(e)
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_df.to_csv("translation_evaluation_results.csv", index=False)

print("\nEvaluation complete! Results saved to translation_evaluation_results.csv")

The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


The repository for meteor contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/meteor.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...


Batch 17 Scores:
BLEU: 0.00
ROUGE-L: 0.00
METEOR: 0.00
BERTScore F1: 0.56


100%|██████████| 18/18 [01:30<00:00,  5.01s/it]


Batch 18 Scores:
BLEU: 0.91
ROUGE-L: 0.00
METEOR: 0.01
BERTScore F1: 0.56

Final Scores for google/mt5-small:
Average BLEU: 0.12
Average ROUGE-L: 0.00
Average METEOR: 0.00
Average BERTScore F1: 0.56

Processing model: ai4bharat/indictrans-v2-en-bn
Error processing model ai4bharat/indictrans-v2-en-bn: ai4bharat/indictrans-v2-en-bn is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

Processing model: ai4bharat/indic-bert


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

Error processing model ai4bharat/indic-bert: Could not load model ai4bharat/indic-bert with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSeq2SeqLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForSeq2SeqLM'>). See the original errors:

while loading with AutoModelForSeq2SeqLM, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py", line 283, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py", line 567, in from_pretrained
    raise ValueError(
ValueError: Unrecognized configuration class <class 'transformers.models.albert.configuration_albert.AlbertConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoder

In [13]:
import pandas as pd
import csv
from datetime import datetime

def save_detailed_results(results, batch_scores, output_file="detailed_translation_results.csv"):
    # Prepare the data for the CSV
    csv_data = []

    # Add a header row
    header = ["Model", "Batch", "BLEU", "ROUGE-L", "METEOR", "BERTScore F1"]
    csv_data.append(header)

    # Add batch scores for each model
    for model, scores in batch_scores.items():
        for batch, score in enumerate(scores, start=1):
            row = [
                model,
                f"Batch {batch}",
                score['BLEU'],
                score['ROUGE-L'],
                score['METEOR'],
                score['BERTScore F1']
            ]
            csv_data.append(row)

        # Add average scores
        avg_scores = next(item for item in results if item["Model"] == model)
        avg_row = [
            model,
            "Average",
            avg_scores['BLEU'],
            avg_scores['ROUGE-L'],
            avg_scores['METEOR'],
            avg_scores['BERTScore_F1']
        ]
        csv_data.append(avg_row)

        # Add an empty row for better readability
        csv_data.append([])

    # Get current timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Add timestamp to the filename
    output_file = f"{timestamp}_{output_file}"

    # Write to CSV
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(csv_data)

    print(f"Detailed results saved to {output_file}")

    # Create a summary DataFrame
    summary_df = pd.DataFrame(results)
    summary_file = f"{timestamp}_summary_results.csv"
    summary_df.to_csv(summary_file, index=False)

    print(f"Summary results saved to {summary_file}")

# Example usage (place this at the end of your main script):
# save_detailed_results(results, batch_scores)

In [11]:
!pip install graphviz
!apt-get install graphviz -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [12]:
from graphviz import Digraph

dot = Digraph(comment='Translation Model Workflow')

# Main workflow
dot.node('A', 'Load Dataset')
dot.node('B', 'Preprocess Data')
dot.node('C', 'Initialize Metrics')
dot.node('D', 'Load Translation Models')
dot.node('E', 'Iterate Through Models')
dot.node('F', 'Batch Processing')
dot.node('G', 'Translate Texts')
dot.node('H', 'Calculate Scores')
dot.node('I', 'Store Results')
dot.node('J', 'More Models?')
dot.node('K', 'Aggregate Results')
dot.node('L', 'Save Detailed CSV')
dot.node('M', 'Generate Report')

# Subgraphs for Metrics and Models
with dot.subgraph(name='cluster_0') as c:
    c.attr(label='Metrics')
    c.node('N', 'BLEU')
    c.node('O', 'ROUGE-L')
    c.node('P', 'METEOR')
    c.node('Q', 'BERTScore')

with dot.subgraph(name='cluster_1') as c:
    c.attr(label='Models')
    c.node('R', 'mBART-large')
    c.node('S', 'OPUS-MT')
    c.node('T', 'NLLB-200')
    c.node('U', 'mT5-small')
    c.node('V', 'IndicTrans-v2')

# Workflow connections
dot.edges(['AB', 'BC', 'CD', 'DE', 'EF', 'FG', 'GH', 'HI', 'IJ'])
dot.edge('J', 'E', label='Yes')
dot.edge('J', 'K', label='No')
dot.edge('K', 'L')
dot.edge('L', 'M')

# Connections to subgraphs
dot.edge('C', 'N')
dot.edge('C', 'O')
dot.edge('C', 'P')
dot.edge('C', 'Q')
dot.edge('D', 'R')
dot.edge('D', 'S')
dot.edge('D', 'T')
dot.edge('D', 'U')
dot.edge('D', 'V')

# Render the graph
dot.render('/content/translation_model_workflow', format='png', cleanup=False)
dot.view('/content/translation_model_workflow')



'/content/translation_model_workflow.pdf'